## **M916 - Machine Learning**
## **Coursework 1**

In [1]:
import numpy as np

In [ ]:
SEED = np.random.seed(7) # set a seed for reproducible results

## **1 Vector and Matrices**

In this section we make a series of vectors's and matrices's operations

In [3]:
# 1 Vector and Matrices

X = np.random.randint(10, size=(3,4))
Y = np.random.randint(10, size=(4,3))

g = np.random.randint(10, size=4)
z = np.random.randint(10, size=4)

In [4]:
print('X:', '\n', X, '\n')
print('Y:', '\n', Y, '\n')
print('g:', '\n', g, '\n')
print('z:', '\n', z, '\n')

X: 
 [[4 9 6 3]
 [3 7 7 9]
 [7 8 9 8]] 

Y: 
 [[7 6 4]
 [0 7 0]
 [7 6 3]
 [5 8 8]] 

g: 
 [7 5 0 0] 

z: 
 [2 8 9 6] 



In [5]:
# inner product (g,z)

inner_g_z = np.dot(g,z)
print('vectors g,z: ', '\n', inner_g_z, '\n')

print('vectors g^T*z: ', '\n', np.dot(g.T, z), '\n')

# matrix - vector

mat_vec_x_g = np.dot(X, g)
print('X, g: ', '\n', mat_vec_x_g, '\n')

# matrix - vector

mat_mat_x_y = np.dot(X, Y)
print('X, Y: ', '\n', mat_mat_x_y)

vectors g,z:  
 54 

vectors g^T*z:  
 54 

X, g:  
 [73 56 89] 

X, Y:  
 [[ 85 147  58]
 [115 181 105]
 [152 216 119]]


In [6]:
# Frobenius norms

xfrob = np.linalg.norm(X, 'fro')
yfrob = np.linalg.norm(Y, 'fro')

print('Frobenius norm of X: ', xfrob)
print('Frobenius norm of Y: ', yfrob)

Frobenius norm of X:  24.24871130596428
Frobenius norm of Y:  19.924858845171276


## **Linear Regression**

In [7]:
import matplotlib.pyplot as plt
import pandas as pd
import sklearn as sk
import seaborn as sns

In [ ]:
# Linear Regression

multivar = '/content/drive/MyDrive/ML/data/multivariate_linear_regression.txt' # insert your own data file path for linear regression

data = pd.read_csv(multivar, header=None)

temp = data.to_numpy()

X = temp[:,:-1]
y = temp[:,-1]

In [9]:
regr = sk.linear_model.LinearRegression()

regr.fit(X, y)

y_pred = regr.predict(X)

In [10]:
print("Bias: ", regr.intercept_)
print("Coefficients: ", regr.coef_)

Bias:  89597.90954279754
Coefficients:  [  139.21067402 -8738.01911233]


In [11]:
lse = np.linalg.inv(X.T @ X) @ X.T @ y

# in case X was not invertible (or the output of this equation could be wrong) we can also verify it with the following equation:
safe_lse = np.linalg.pinv(X) @ y

if lse.all() != safe_lse.all():
    print('Least Squares estimates with pseudo-inverse: ', safe_lse)
else:
    print('Least Squares estimates: ', lse)

# 'lse found at: https://pythonnumericalmethods.studentorg.berkeley.edu/notebooks/chapter16.04-Least-Squares-Regression-in-Python.html'

Least Squares estimates:  [  140.86108621 16978.19105903]


## **The Boston Housing Dataset**


In [12]:
path = '/content/drive/MyDrive/ML/HousingData.csv'

house = pd.read_csv(path)

house.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,NaN,36.2


In [13]:
temp = house.to_numpy()

X = temp[:,:-1]
y = temp[:,-1]


X = np.column_stack((np.ones((X.shape[0])),X))

# **Split and clean data**

In [ ]:
def clean_data_split(array, test_size, random_state):
    """
    A function that seperates train from test data when taken as a unified array.
    Args:
    array: An array that contains all data (train and test).
    test_size: Define the size of the test set. Takes a float (i.e. 0.1 = 10% test size).
    random_state: set an integer as a random state for same split (reproducibility)
    """
    X_train, X_test = sk.model_selection.train_test_split(array, test_size=test_size, random_state=random_state)
    x_train = pd.DataFrame(X_train).interpolate(method='linear')
    x_test = pd.DataFrame(X_test).interpolate(method='linear')
    x_train = x_train.bfill().ffill()
    x_test = x_test.bfill().ffill()
    return x_train, x_test

# found this method (.interpolate()) here: https://saturncloud.io/blog/python-pandas-how-to-remove-nan-and-inf-values/
# it seemed more 'fair' (and more effective) regarding data manipulation (criteria to remove nan values).
# also found: https://stackoverflow.com/questions/73879092/why-does-not-pandas-interpolate-method-works-when-having-the-consecutive-nan-val

# **90/10**

In this section begins a series of test splits of the data to find an ideal split of the data. First, we have a 90% train set vs 10% test set split of data.

In [ ]:
def train(test_size):
    """
    A function that utilizes a split function of data and prints the coefficients and the Mean Square Error (MSE) metric on the dataset when calculated with a linear regression model.
    Args:
    test_size: takes a float as an input, which indicates the size(percentage) of the test set.
    """
    y_train, y_test = clean_data_split(y, test_size, 7)
    y_train = y_train.to_numpy()
    y_test = y_test.to_numpy()

    x_train, x_test = clean_data_split(X, test_size, 7)
    X_train = x_train.to_numpy()
    X_test = x_test.to_numpy()


    beta_train = np.linalg.pinv(X_train.T @ X_train) @ X_train.T @ y_train


    intercept_train = beta_train[0]
    coef_train = beta_train[1:]
    # print('Intercept train: ', intercept_train)
    # print('Betas train: ', '\n', coef_train)


    # Evaluation (train)
    y_pred_train = X_train @ beta_train

    print('Coefficient of determination: ', sk.metrics.r2_score(y_train, y_pred_train))
    print('Mean squared error: ', sk.metrics.mean_squared_error(y_train, y_pred_train))


    # Evaluation (test)
    y_pred_test = X_test @ beta_train


    print('Coefficient of determination (test): ', sk.metrics.r2_score(y_test, y_pred_test))
    print('Mean squared error (test): ', sk.metrics.mean_squared_error(y_test, y_pred_test))

In [16]:
train(0.1)

Coefficient of determination:  0.726997243410358
Mean squared error:  23.62949155221228
Coefficient of determination (test):  0.7215549378217503
Mean squared error (test):  17.890813571122177


Judging from the outcome of $R^2$ and MSE  metrics:

**Coefficient of determination:**  0.726997243410358

**Mean squared error:**  23.62949155221228

**Coefficient of determination (test):**  0.7215549378217503

**Mean squared error (test):**  17.890813571122177

this split seems to work efficiently, providing an almost equal evaluation between training and test set. However, the small size of the test set is quite concerning as we don't have enough data to evaluate its performance. Consequently, a split with such a small test set and so big train set could even "mask" a quite possible overfitting.

# **80/20**

In [17]:
train(0.2)

Coefficient of determination:  0.7608316997490375
Mean squared error:  20.408541875458287
Coefficient of determination (test):  0.5508567551564008
Mean squared error (test):  36.29358586151897


The results of this training size are quite similar with the previous one. However, the lower $R^2$ score in test set indicates a possible weakness to generalization, meaning that we might actually have overfited our data. This surely means that a lower train set is a more optimal solution.

# **70/30**

In [18]:
train(0.3)

Coefficient of determination:  0.7533868731789164
Mean squared error:  21.940224991640388
Coefficient of determination (test):  0.6242726639256315
Mean squared error (test):  27.728679841653886


In this instance we obtain a better result than the previous one: test set has a score closer to the training set. Even though the test evaluation results are not the best in all the possible splits (i.e. 90/10), it seems as a more "natural" split in general: while training set size is quite sufficient, the model seems able to generalize in order to predict a good outcome while also having a sufficient test set to evaluate clearly. Furthermore, in a ratio as this one, I think that we don't have to worry about an overfitting scenario anymore.

# **50/50**

In [19]:
train(0.5)

Coefficient of determination:  0.792333420318353
Mean squared error:  17.010454731884337
Coefficient of determination (test):  0.5931297653705667
Mean squared error (test):  35.14202346754178


The last instance provides us with some unexpected results: while the test metrics are quite expected (underfitting), we have the best results in training, better than the ones with bigger training size. This might happen due to homogenous data (maybe interpolation played a part there too), which led to an ocassion like this. To put it differently, train set was probably small enough to fit just right, but the rest of the data (test set) could not be predicted. All in all, this seems more like a "risky" split of the data, an approach that will probably not return good results in train especially in more complex datasets/scenarios.